# Annotations

In [1]:
from ipynb.fs.full.koselleck import *

In [2]:
ANNOCOLS = ['Changes','From','To','Notes','is_valid']

In [3]:
def start_sheet_changes(min_peaks=1):
    dfchangepoints=get_signif_novelty_scores(min_peaks=min_peaks).sort_values('foote_novelty_z',ascending=False).drop_duplicates('word',keep='first').set_index('word')
    dfchangepoints=dfchangepoints.join(get_all_novelty_scores().groupby('word').mean(),rsuffix='_avg_sign')
    dfchangepoints=dfchangepoints.join(get_all_novelty_scores().groupby('word').mean(),rsuffix='_avg')
    dfchangepoints = dfchangepoints.sort_values('foote_novelty_z_avg_sign',ascending=False)
    dfchangepoints['rank'] = dfchangepoints.foote_novelty_z_avg_sign.rank(ascending=False).apply(int)
    dfchangepoints = dfchangepoints.rename({'period':'changepoint'},axis=1)
    for ac in ANNOCOLS: dfchangepoints[ac]=''
    prefcols=['rank','changepoint'] + ANNOCOLS + ['word_num_peaks','foote_novelty_z_avg_sign']
    dfchangepoints = dfchangepoints[prefcols + sorted([c for c in dfchangepoints.columns if c not in set(prefcols)])]
    return round(dfchangepoints,2)

In [4]:
# odf=get_changeranks_sattelzeit().join(start_sheet_changes())
# odf

In [5]:
# read_sheet()

In [20]:
def update_sheet_changes(df=None):
    df=start_sheet_changes() if df is None else df
    df_now=read_sheet()
    if len(df_now):
        for col in ANNOCOLS:
            df[col]=df_now[col]
    df=df.fillna('')
    prefcols=['rank_word']
    sufcols=['lnm_perc_word','lnm_perc','lnm_z','lnm']
    pref=prefcols + ANNOCOLS + sufcols
    df=df[pref + [c for c in df.columns if c not in set(pref)]]
    df.rank_word=df.rank_word.apply(int)
    df=round(df,2)
    df=df.sort_values('rank_word')
    write_sheet(df)
    return df

In [6]:
# update_sheet_changes(odf)

## Gather data from anno

In [7]:
dfanno=read_sheet()

In [16]:
DFANNO=None
def get_dfanno(force=False,dbkey='dfanno'):
    global DFANNO
    if not force:
        if DFANNO is not None: return DFANNO
        with get_veclib('anno') as vl:
            if dbkey in vl: return vl[dbkey]
    print('Reading annotations from online')
    DFANNO=df=read_sheet()
    with get_veclib('anno') as vl:
        vl[dbkey]=df
        vl.commit()
    return DFANNO

In [18]:
dfanno=get_dfanno(force=False)
dfanno

,rank_word,Changes,From,To,Notes,is_valid,lnm_perc_word,lnm_perc,lnm_z,lnm,...,is_signif_avg,is_signif_avg_sign,p_peak,p_peak_avg,p_peak_avg_sign,p_trough,p_trough_avg,p_trough_avg_sign,period_avg,period_avg_sign
word,,,,,,,,,,,,,,,,,,,,,
greed,1,,,,,n,100,99.99,10.46,1.42,...,,,,,,,,,,
maladies,7018,,,,,,37.75,36.97,-0.4,0.05,...,,,,,,,,,,
barking,7019,,,,,,37.74,36.36,-0.4,0.05,...,,,,,,,,,,
dwarf,7020,,,,,,37.73,36.64,-0.4,0.05,...,,,,,,,,,,
uncouth,7021,,,,,,37.72,36.64,-0.4,0.05,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rotation,7900,,,,,,29.92,29.47,-0.46,0.05,...,0.04,0.04,0.04,0.92,0.92,1,0.95,0.95,1806.2,1806.2
lamentable,2444,,,,,,78.33,74.3,0.07,0.11,...,0.05,0.05,0.03,0.91,0.91,1,0.71,0.71,1822.5,1822.5
bullion,6764,,,,,,40,38.18,-0.39,0.06,...,0.05,0.05,0.02,0.81,0.81,1,0.9,0.9,1792.5,1792.5


In [20]:
def get_words_anno_valid():
    dfanno=get_dfanno()
    return dfanno[dfanno.is_valid.str.startswith('y')].index
def get_words_anno_invalid():
    dfanno=get_dfanno()
    return dfanno[dfanno.is_valid.str.startswith('n')].index
        

In [24]:
len(get_words_anno_valid())

77

In [42]:
def _respell_change(x):
    x=x.replace('z','s')
    x
    return x

def get_changes_anno(min_count=3):
    odf=pd.DataFrame([
        {'word':w, 'Change':cx.strip().title(), **dict(row)}
        for w,row in get_dfanno().query('Changes!=""').iterrows()
        for cx in row.Changes.split(',')
    ])
    odf.Change=odf.Change.apply(_respell_change)
    if min_count: odf=odf.groupby('Change').filter(lambda x: len(x)>=min_count)
    return odf

In [46]:
# get_changes_anno()